## Reading Dataset

In [2]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
'''
Torchvision : Python imaging format image를 불러옴
transforms.Compose : 전처리 pipeline
transforms.ToTensor : 이미지(range [0 , 255])를 float텐서(shape(CxHxW) with range[0.0, 1.0]) 형태로 뱐환
PIL image (H x W x C) -> Tensor (C x H x W)
transforms.Normalize : input = (input - 0.5 ) / 0.5
'''

# CIFAR10 data를 transform 시켜서 불러옴
trainset = torchvision.datasets.CIFAR10(root='./',
                                        train=True,
                                        download=False,
                                        transform=transform)


testset = torchvision.datasets.CIFAR10(root='./',
                                       train=False,
                                       download=False,
                                       transform=transform)



In [3]:
from torch.utils.data import DataLoader

'''
torch.utils.data.DataLoader : data를 batch size로 묶음
'''

trainloader = DataLoader(trainset, 
                         batch_size=10,
                         shuffle=True, 
                         drop_last=False)

testloader = DataLoader(testset, 
                        batch_size=10,
                        shuffle=False,
                        drop_last=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
trainiter = iter(trainloader)
images, labels = trainiter.next()

print(len(trainloader))
print(images.shape)
print(labels)

5000
torch.Size([10, 3, 32, 32])
tensor([3, 0, 9, 1, 5, 9, 7, 4, 6, 7])


## Modeling

In [5]:
#config
src = {'input_size':3*32*32,
       'hidden_size':50,
       'output_size':10,
       'init_weight_range':0.5,
       'num_epochs':20,
       'batch_size':10,
       'learning_rate':1e-3}

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, src):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(src['input_size'], src['hidden_size'])
        self.fc2 = nn.Linear(src['hidden_size'], src['output_size'])
                             
        ## sequential layer
        '''
        self.seq_fc = nn.Sequential(
                            nn.Linear(src['input_size'], src['hidden_size']),
                            nn.Linear(src['hidden_size'], src['output_size'])
                            )
        '''
        self.init_range = src['init_weight_range']
    
    #가중치 초기화
    def init_weight(self): 
        self.fc1.weight.data.uniform_(-self.init_range, self.init_range)
        self.fc2.weight.data.uniform_(-self.init_range, self.init_range)
        '''
        for fc in self.seq_fc:
            fc.weight.data.uniform_(-self.init_range, self.init_range)
        '''
    def forward(self, img):
        x = img.view(img.shape[0], -1)
        #--------------------
        x = self.fc1(x)
        x = F.relu(x) #relu
        x = self.fc2(x)
        y = F.softmax(x, dim=0) #softmax
        #--------------------
        
        ## 4 lines above are identical to 
        '''
        y = self.seq_fc(x)
        '''
        return y

In [7]:
model = Net(src)
y = model(images)

## Training and Evaluating

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      src['learning_rate'], 
                      momentum=0.9)

In [9]:
from tqdm import tqdm

model.init_weight()

for epoch in tqdm(range(src['num_epochs'])):
    current_loss = 0.0
#     model.train(True)
    
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        step = i + 1
        current_loss += loss.item()
        
        if step % 1000 == 0 and step != 0:     # print every 1000 mini-batches
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %
                  (epoch + 1, src['num_epochs'], step, len(trainloader)//1000 * 1000, current_loss / 1000))
            current_loss = 0.0 #천번동안 있었던 loss의 누적값 평균, 그 후 reset

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

Epoch [1/20], Step [1000/5000], Loss: 2.3155
Epoch [1/20], Step [2000/5000], Loss: 2.2938
Epoch [1/20], Step [3000/5000], Loss: 2.2681
Epoch [1/20], Step [4000/5000], Loss: 2.2455
Epoch [1/20], Step [5000/5000], Loss: 2.2270


  5%|████▏                                                                              | 1/20 [00:47<14:58, 47.30s/it]

Epoch [2/20], Step [1000/5000], Loss: 2.2229
Epoch [2/20], Step [2000/5000], Loss: 2.2120
Epoch [2/20], Step [3000/5000], Loss: 2.2046
Epoch [2/20], Step [4000/5000], Loss: 2.1932
Epoch [2/20], Step [5000/5000], Loss: 2.1890


 10%|████████▎                                                                          | 2/20 [01:57<16:14, 54.16s/it]

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [19:28<02:21, 70.56s/it]

Epoch [19/20], Step [1000/5000], Loss: 2.0916
Epoch [19/20], Step [2000/5000], Loss: 2.1023
Epoch [19/20], Step [3000/5000], Loss: 2.0971
Epoch [19/20], Step [4000/5000], Loss: 2.0963
Epoch [19/20], Step [5000/5000], Loss: 2.0936


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [20:27<01:07, 67.25s/it]

Epoch [20/20], Step [1000/5000], Loss: 2.0889
Epoch [20/20], Step [2000/5000], Loss: 2.0976
Epoch [20/20], Step [3000/5000], Loss: 2.0888
Epoch [20/20], Step [4000/5000], Loss: 2.0978
Epoch [20/20], Step [5000/5000], Loss: 2.0899


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [21:28<00:00, 65.16s/it]


In [11]:
import torch
# Test the Model
correct = 0
total = 0
for i, data in enumerate(testloader):
    inputs, labels = data
#     images = images.view(-1, 28*28)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.shape[0]
    correct += (predicted == labels).sum()

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

# Accuracy of the network on the test images: 35 %

Accuracy of the network on the test images: 35 %
